<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparação-dos-Dados" data-toc-modified-id="Preparação-dos-Dados-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preparação dos Dados</a></span><ul class="toc-item"><li><span><a href="#8.1---SVM" data-toc-modified-id="8.1---SVM-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>8.1 - SVM</a></span></li></ul></li></ul></div>

# Preparação dos Dados

In [1]:
# Importando módulos necessários - Tratamento dos Dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tarfile
from six.moves import urllib

# Importando módulos necessários - Preparação de Dados
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import Imputer
from future_encoders import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from future_encoders import ColumnTransformer
from sklearn.pipeline import Pipeline

# Importando módulos necessários - Treinamento
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Importando módulos necessários - Métricas e Validações
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

# Importando módulos necessários - Sintonizando e Salvando o Modelo
from sklearn.model_selection import GridSearchCV
from scipy.stats import geom, expon
from sklearn.externals import joblib

Após a importação dos módulos, a primeira coisa a se fazer é ler o arquivo e **separar** o conjunto em datasets de treino e de teste. 

In [2]:
# Definindo variáveis e função para extração do dataset
DOWNLOAD_ROOT = 'https://raw.githubusercontent.com/ageron/handson-ml/master/' # Define url root onde encontram-se os arquivos
HOUSING_PATH = os.path.join('datasets', 'housing') # Pasta onde será(ão) organizado(s) o(s) arquivo(s)
HOUSING_URL = DOWNLOAD_ROOT + 'datasets/housing/housing.csv' # Parte específica da url para baixar arquivo específico

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    """
    Dada uma URL, esta função é responsável por realizar o download, extração e organização de datasets automaticamente
    
    INPUTS: 
    housing_url = URL no qual se encontra o arquivo a ser baixado.
    housing_path = caminho dentro do SO onde o arquivo será armazenado.
    """
    if not os.path.isdir(housing_path): # Se não há pasta com o nome específicado, cria uma no SO.
        os.makedirs(housing_path) 
    if 'tgz' in housing_url: # Se trata-se de um arquivo .tgz (Linux)
        try: 
            filename = 'housing'
            extension = '.tgz'
            tgz_path = os.path.join(housing_path, filename+extension)
            urllib.request.urlretrieve(housing_url, tgz_path)
            housing_tgz = tarfile.open(tgz_path)
            housing_tgz.extractall(path=housing_path)
            housing_tgz.close
        except tarfile.ReadError:
            print('Erro de Leitura: Não foi possível abrir o arquivo.')
        except urllib.error.URLError:
            print('Erro HTTP: URL não encontrada (Erro 404).')
    elif 'csv' in housing_url: # Se trata-se de um arquivo .csv
        try:
            filename = 'housing' # Nome do arquivo
            extension = '.csv' # Extensão
            csv_path = os.path.join(housing_path, filename+extension) # Mapeando caminho no SO
            urllib.request.urlretrieve(housing_url, csv_path) # Acessa link para download
        except urllib.error.URLError:
            print('Erro HTTP: URL não encontrada (Erro 404).')
        else:
            print(f'Download realizado com sucesso! Arquivo pode ser encontrado em: \n{housing_path+"/"+filename+extension}')

# Chamando função
fetch_housing_data()

# Função para leitura de arquivo
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, 'housing.csv')
    return pd.read_csv(csv_path)

# Lendo arquivo
housing = load_housing_data()

# Verificando cabeçalho
housing.head()

Download realizado com sucesso! Arquivo pode ser encontrado em: 
datasets\housing/housing.csv


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
# Criando nova categoria
housing['income_cat'] = np.ceil(housing['median_income'] / 1.5) # Limitando valores
housing['income_cat'].where(housing['income_cat'] < 5, 5.0, inplace=True)

# Separando dataset
split = StratifiedShuffleSplit(n_splits=1, test_size=.2, random_state=42)
for train_index, test_index in split.split(housing, housing['income_cat']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

# Comparando
print(f'Shape do dataset original: {housing.shape}')
print(f'Shape de strat_train_set: {strat_train_set.shape}')
print(f'Shape de strat_test_set: {strat_test_set.shape}')

# Excluindo colunas adicionais
for set_ in (strat_train_set, strat_test_set):
    set_.drop('income_cat', axis=1, inplace=True)
    
# Criando cópia
housing = strat_train_set.drop('median_house_value', axis=1)
housing_labels = strat_train_set['median_house_value'].copy()

print(f'Shape de housing modificado: {housing.shape}')
print(f'Shape de housing_labels: {housing_labels.shape}')

Shape do dataset original: (20640, 11)
Shape de strat_train_set: (16512, 11)
Shape de strat_test_set: (4128, 11)
Shape de housing modificado: (16512, 9)
Shape de housing_labels: (16512,)


In [4]:
# Classe responsável por combinar alguns atributos do dataset

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinadorAtributos(BaseEstimator, TransformerMixin):
    """
    Classe criada para combinar atributos e criar novas features de acordo com as informações contidas no Dataset.
    Atributos criados:
        - 
    """
    def __init__(self, add_bedrooms_per_room=True, add_rooms_per_population=True):
        """
        Método construtor: define hiperparâmetros, neste caso, combinações entre atributos diferentes
            - add_bedrooms_per_room
            - add_rooms_per_population
        """
        self.add_bedrooms_per_room = add_bedrooms_per_room
        self.add_rooms_per_population = add_rooms_per_population
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            if self.add_rooms_per_population:
                rooms_per_population = X[:, rooms_ix] / X[:, population_ix]
                return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room,
                            rooms_per_population]
            else:
                return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        if self.add_rooms_per_population:
            rooms_per_population = X[:, rooms_ix] / X[:, population_ix]
            return np.c_[X, rooms_per_household, population_per_household, rooms_per_population]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

<font size=5> **APLICANDO PIPELINE** </font>

In [5]:
# Definições iniciais 
num_attribs = list(housing.drop('ocean_proximity', axis=1))
cat_attribs = ['ocean_proximity']

# Definindo Pipeline numérico
num_pipeline = Pipeline([
    ('imputer', Imputer(strategy="median")),
    ('attribs_adder', CombinadorAtributos()),
    ('std_scaler', StandardScaler()),
])

# Executando todo o Pipeline
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(sparse=False), cat_attribs)
])

# Definindo dados preparados
housing_prepared = full_pipeline.fit_transform(housing)

# Comparando
print(f'Shape housing original: {housing.shape}')
print(f'Shape housing_prepared: {housing_prepared.shape}')

Shape housing original: (16512, 9)
Shape housing_prepared: (16512, 17)


## 8.1 - SVM

Try a Support Vector Machine regressor (```sklearn.svm.SVR```) wth various hyperparameters such as ```kernel=linear``` (with various values for the C hyperparameter) or ```kernel="rbf"``` (with various values for the C and ```gamma``` hyperparameters). How does the best SVR predictor perform?

In [7]:
# Importando algorítimo SVM
from sklearn.svm import SVR

In [10]:
# Treinando algorítimo sem nenhuma especificação
svm_reg = SVR()
svm_reg.fit(housing_prepared, housing_labels)
svm_reg.predict(housing_prepared)

array([179160.75917566, 179940.53375474, 179191.67464573, ...,
       179299.85241107, 179594.29436694, 179564.67686674])

In [12]:
# Realizando predições com a totalidade do conjunto de treinamento
housing_predictions = svm_reg.predict(housing_prepared)

# Calculando o erro RMSE (sem raíz quadrada)
svm_mse = mean_squared_error(housing_labels, housing_predictions)

# Aplicando raíz quadrada
svm_rmse = np.sqrt(svm_mse)
svm_rmse

118573.30094001174

In [24]:
# Criando com os demais algorítimos em um pipeline
def model_train(df_prepared, df_labels, 
                lin_reg=False, svm_reg=False, tree_reg=False, forest_reg=False):
    """
    Função que recebe dados de entrada e um algorítimo selecionado para realizar o treinamento completo
    """
    # Verificando o modelo escolhido
    if lin_reg:
        model = LinearRegression()
        desc = 'Linear Regression'
    elif svm_reg:
        model = SVR()
        desc = 'SVM'
    elif tree_reg:
        model = DecisionTreeRegressor()
        desc = 'Decision Tree Regressor'
    elif forest_reg:
        model = RandomForestRegressor()
        desc = 'Random Forest Regressor'
    else:
        print('Nenhum modelo selecionado.')
        return
    
    # Treinando e computando erro
    model.fit(df_prepared, df_labels)
    model_predictions = model.predict(df_prepared)
    model_mse = mean_squared_error(df_labels, model_predictions)
    model_rmse = np.sqrt(model_mse)
    print(f'Erro Médio Quadrádico para {desc}: {model_rmse:.2f}')

In [26]:
model_train(housing_prepared, housing_labels, lin_reg=True)
model_train(housing_prepared, housing_labels, svm_reg=True)
model_train(housing_prepared, housing_labels, tree_reg=True)
model_train(housing_prepared, housing_labels, forest_reg=True)

Erro Médio Quadrádico para Linear Regression: 67324.65
Erro Médio Quadrádico para SVM: 118573.30
Erro Médio Quadrádico para Decision Tree Regressor: 0.00
Erro Médio Quadrádico para Random Forest Regressor: 22174.99
